In [1]:
from pyesasky.pyesasky import ESASkyWidget
from pyesasky.pyesasky import Catalogue
from pyesasky.pyesasky import CatalogueDescriptor
from pyesasky.pyesasky import MetadataDescriptor
from pyesasky.pyesasky import MetadataType

In [2]:
# instantiating pyESASky instance
esasky = ESASkyWidget()

In [3]:
# loading pyESASky instance
esasky

ESASkyWidget(layout=Layout(align_self='stretch', height='400px'))

In [4]:
#####################################################
# EX.1 creating a user defined catalogue on the fly #
#####################################################
catalogue = Catalogue('test catalogue name', 'equatorial', '#ee2345', 10)

coordinates frame equatorial not recognized. Possible options are J2000 and Galactic. Applied J2000 by default.


In [5]:
# adding sources to the catalogue
catalogue.addSource('source name A', '105.69239256', '-8.45235969', None, [{"name":"Flux 1", "value":"10.5", "type":"STRING" },{"name":"Flux 2", "value":"1.7", "type":"STRING" }])
catalogue.addSource('source name B', '106.70779763', '-8.31350997', None, [{"name":"Flux 1", "value":"11.5", "type":"STRING" },{"name":"Flux 2", "value":"2.7", "type":"STRING" }])
catalogue.addSource('source name c', '107.70779763', '-8.31350997', None, [{"name":"Flux 1", "value":"12.5", "type":"STRING" },{"name":"Flux 2", "value":"0.7", "type":"STRING" }])

In [6]:
# overlay catalogue in pyESASky
esasky.overlayCatalogueWithDetails(catalogue)

In [7]:
############################################
# EX.2 importing a catalogue from CSV file #
############################################
# CatalogueDescriptor('<catName>', '<HTMLcolor>', <lineWidth>, '<idColumn>', '<nameColumn>', '<RAColumn>', '<DecColumn>', Metadata)
# where:
# - <catName> : name of the catalogue that will be used in pyESASky as label
# - <HTMLcolor> : HTML color. It could be a "Color name", "Hex color code" or "RGB color code"
# - <lineWidth> : width used to draw sources. From 1 to 10
# - <idColumn> : name of the column containing a unique identifier for sources if any. None if not applicable
# - <nameColumn> : name of the column with the name of the source
# - <RAColumn> : name of the RA column in degrees
# - <DecColumn> : name of the Dec column in degrees
#  - Metadata : list of pyesasky.pyesasky.MetadataDescriptor in case it has been defined. [] otherwise.
catalogueDesc =CatalogueDescriptor('my test', 'red', 5, 'id', 'name', 'ra', 'dec', [])

In [8]:
# parse, import and overlay a catalogue from a CSV
esasky.overlayCatalogueFromCSV('/home/fgiordano/testcat', ',', catalogueDesc)

Column identified: #id, name, ra, dec, flux1, flux2
{name} column identified: name
{centerRaDeg} column identified: ra
{currDecDeg} column identified: dec
Processed 4 lines.


In [9]:
###################################################################
# EX.3 importing a catalogue from AstropyTable using Gaia archive #
###################################################################
from astroquery.gaia import Gaia

Created TAP+ (v1.0.1) - Connection:
	Host: gea.esac.esa.int
	Use HTTPS: False
	Port: 80
	SSL Port: 443


In [10]:
job = Gaia.launch_job("select top 10\
 ra, dec, source_id, designation, ref_epoch,ra_dec_corr,astrometric_n_obs_al,matched_observations,duplicated_source,phot_variable_flag \
 from gaiadr2.gaia_source order by source_id", verbose=True)


Launched query: 'select top 10 ra, dec, source_id, designation, ref_epoch,ra_dec_corr,astrometric_n_obs_al,matched_observations,duplicated_source,phot_variable_flag  from gaiadr2.gaia_source order by source_id'
200 200
[('Date', 'Mon, 06 May 2019 14:42:46 GMT'), ('Server', 'Apache/2.4.6 (Red Hat Enterprise Linux) OpenSSL/1.0.2k-fips PHP/5.4.16 mod_jk/1.2.43'), ('Cache-Control', 'no-cache, no-store, max-age=0, must-revalidate'), ('Pragma', 'no-cache'), ('Expires', '0'), ('X-XSS-Protection', '1; mode=block'), ('X-Frame-Options', 'SAMEORIGIN'), ('X-Content-Type-Options', 'nosniff'), ('Set-Cookie', 'JSESSIONID=3EE0D1A96672B71BA50EC5143A4EB1D3; Path=/tap-server; HttpOnly'), ('Content-Encoding', 'gzip'), ('Content-Disposition', 'attachment;filename="1557153766796O-result.vot"'), ('Transfer-Encoding', 'chunked'), ('Content-Type', 'application/x-votable+xml')]
Retrieving sync. results...
Query finished.


In [11]:
myGaiaData = job.get_results()
print(myGaiaData)

        ra                 dec          ... duplicated_source phot_variable_flag
       deg                 deg          ...                                     
------------------ -------------------- ... ----------------- ------------------
44.996153684159594 0.005615806210679649 ...             False      NOT_AVAILABLE
45.004316164207644 0.021045032689712983 ...             False      NOT_AVAILABLE
  45.0049742449841 0.019877000365797714 ...             False      NOT_AVAILABLE
 44.99503703932583  0.03815183599451371 ...             False      NOT_AVAILABLE
 44.96389532530429 0.043595184822725674 ...             False      NOT_AVAILABLE
 44.99832632511778   0.0663337657163784 ...             False      NOT_AVAILABLE
 45.04828219158795   0.0482562232180315 ...             False      NOT_AVAILABLE
 45.02361786160386  0.06842024221607944 ...             False      NOT_AVAILABLE
 45.02672638277647  0.08169949145380392 ...             False      NOT_AVAILABLE
 45.03908018542775  0.086854

In [12]:
job.get_data()

ra,dec,source_id,designation,ref_epoch,ra_dec_corr,astrometric_n_obs_al,matched_observations,duplicated_source,phot_variable_flag
deg,deg,,,yr,,,,,
float64,float64,int64,object,float64,float32,int32,int16,bool,object
44.996153684159594,0.005615806210679649,4295806720,Gaia DR2 4295806720,2015.5,0.022670548,107,13,False,NOT_AVAILABLE
45.004316164207644,0.021045032689712983,34361129088,Gaia DR2 34361129088,2015.5,0.06490505,97,11,False,NOT_AVAILABLE
45.0049742449841,0.019877000365797714,38655544960,Gaia DR2 38655544960,2015.5,0.11690165,104,12,False,NOT_AVAILABLE
44.99503703932583,0.03815183599451371,309238066432,Gaia DR2 309238066432,2015.5,0.042778816,114,13,False,NOT_AVAILABLE
44.96389532530429,0.043595184822725674,343597448960,Gaia DR2 343597448960,2015.5,0.095711425,114,13,False,NOT_AVAILABLE
44.99832632511778,0.0663337657163784,515396233856,Gaia DR2 515396233856,2015.5,0.56088775,105,12,False,NOT_AVAILABLE
45.04828219158795,0.0482562232180315,549755818112,Gaia DR2 549755818112,2015.5,-0.0028029205,114,13,False,NOT_AVAILABLE
45.02361786160386,0.06842024221607944,828929527040,Gaia DR2 828929527040,2015.5,0.11152559,104,12,False,NOT_AVAILABLE


In [13]:
# overlayCatalogueFromAstropyTable('<catName>', '<cooFrame>', <HTMLcolor>, '<(astropy.table)>', '<RAColumn>', '<DecColumn>', '<nameColumn>')
# where:
# - <catName> : name of the catalogue that will be used in pyESASky as label
# - <HTMLcolor> : HTML color. It could be a "Color name", "Hex color code" or "RGB color code"
# - <lineWidth> : width used to draw sources. From 1 to 10
# - <idColumn> : name of the column containing a unique identifier for sources if any. None if not applicable
# - <nameColumn> : name of the column with the name of the source
# - <RAColumn> : name of the RA column in degrees
# - <DecColumn> : name of the Dec column in degrees

esasky.overlayCatalogueFromAstropyTable('Gaia DR2', 'J2000', '#a343ff', 5, myGaiaData, '','','')

-----------
ra
pos.eq.ra;meta.main
-----------
dec
pos.eq.dec;meta.main
-----------
source_id
meta.id
-----------
designation
meta.id;meta.main
-----------
ref_epoch
meta.ref;time.epoch
-----------
ra_dec_corr
stat.correlation;pos.eq.ra;pos.eq.dec
-----------
astrometric_n_obs_al
meta.number
-----------
matched_observations
meta.number
-----------
duplicated_source
meta.code.status
-----------
phot_variable_flag
meta.code;src.var
